<a href="https://colab.research.google.com/github/HARASEON/Deep_Learning_tensorflow/blob/main/Day3_Data_augmentation_horses_vs_humans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Augmentation
GAN(Generative Adversarial Networks) by Goodfellow
* 생성하는 딥러닝 모델
* 데이터로 표현할 수 있는 모든 것을 생성할 수 있음
* 실제와 유사한 형태로 생성
* 없는 데이터를 채울 때
* Super-Resolution: 저화질을 고화질로
* Image-to-Image translation

DCGAN(Generative Adversarial Networks)-Generator, Descriminator

* Goal: val_loss < 0.6

In [ ]:
import tensorflow as tf
import urllib
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D,Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Data loading
_TRAIN_URL = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
_TEST_URL = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"
urllib.request.urlretrieve(_TRAIN_URL, 'horse-or-human.zip')
local_zip = 'horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/horse-or-human/')
zip_ref.close()
urllib.request.urlretrieve(_TEST_URL, 'validation-horse-or-human.zip')
local_zip = 'validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/validation-horse-or-human/')
zip_ref.close()

# Data Generation
train_datagen = ImageDataGenerator(
        rescale=1 / 255,
        rotation_range=40,        # degree range for random rotations
        width_shift_range=0.2,    # fraction of total width
        height_shift_range=0.2,   # fraction of total height
        shear_range=0.2,          # shear angle in counter-clockwise direction
        zoom_range=0.2,           # zoom 
        horizontal_flip=True,
        fill_mode='nearest'
        )
validation_datagen = ImageDataGenerator(rescale = 1./255) 

TRAINING_DIR =  'tmp/horse-or-human/'
train_generator = train_datagen.flow_from_directory(
        TRAINING_DIR, 
        target_size=(300, 300), 
        class_mode='binary' # catagorical for more than 2
       )

VALIDATION_DIR = 'tmp/validation-horse-or-human/'
validation_generator = validation_datagen.flow_from_directory(
        VALIDATION_DIR, 
        target_size=(300, 300), 
        class_mode='binary',
        )

# Model building
model = tf.keras.models.Sequential([
        Conv2D(16, (3, 3), input_shape=(300, 300, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.5),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(), 
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

# Model fitting
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, 
                             save_best_only=True, 
                             save_weights_only=True, 
                             monitor='val_loss', 
                             verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit( train_generator, 
        validation_data=(validation_generator),
        epochs=50,
        callbacks=[checkpoint, early_stopping]
    )
model.load_weights(checkpoint_path) 
model.save("horse-human(gen).h5")

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/50
33/33 [==============================] - ETA: 0s - loss: 0.6961 - accuracy: 0.4995
Epoch 1: val_loss improved from inf to 0.69208, saving model to my_checkpoint.ckpt
33/33 [==============================] - 40s 805ms/step - loss: 0.6961 - accuracy: 0.4995 - val_loss: 0.6921 - val_accuracy: 0.5000
Epoch 2/50
33/33 [==============================] - ETA: 0s - loss: 0.6911 - accuracy: 0.5394
Epoch 2: val_loss improved from 0.69208 to 0.66939, saving model to my_checkpoint.ckpt
33/33 [==============================] - 26s 794ms/step - loss: 0.6911 - accuracy: 0.5394 - val_loss: 0.6694 - val_accuracy: 0.6367
Epoch 3/50
33/33 [==============================] - ETA: 0s - loss: 0.6418 - accuracy: 0.6310
Epoch 3: val_loss did not improve from 0.66939
33/33 [==============================] - 26s 789ms/step - loss: 0.6418 - accuracy: 0.6310 - val_loss: 0.7461 - val_accuracy: 0.5469
Epoch 4/50
33/33 [====